In [1]:
from __future__ import print_function

import os
import json
import codecs
import cProfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.datasets import make_blobs

from pyspark import SparkContext
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler

from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

%matplotlib inline

In [2]:
MONGO_URL_INPUT = "mongodb://192.168.0.20:27017/yelp.review?ssl=false"
MONGO_URL_OUTPUT = "mongodb://192.168.0.20:27017/yelp.teste"

spark = SparkSession.builder \
    .appName("kmeans-spark") \
    .master("spark://spark:7077") \
    .config("spark.mongodb.input.uri", MONGO_URL_INPUT) \
    .config("spark.mongodb.output.uri", MONGO_URL_OUTPUT) \
    .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.12:3.0.0')\
    .getOrCreate()
sqlContext = SQLContext(spark)

In [5]:
def getDataFrame():
    return spark.read.format("com.mongodb.spark.sql.DefaultSource").load()

cProfile.run('getDataFrame()')

         602 function calls (592 primitive calls) in 30.837 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   30.837   30.837 <ipython-input-5-4b4f12b62019>:1(getDataFrame)
        1    0.000    0.000   30.837   30.837 <string>:1(<module>)
       24    0.000    0.000    0.000    0.000 __init__.py:1286(debug)
       24    0.000    0.000    0.000    0.000 __init__.py:1530(getEffectiveLevel)
       24    0.000    0.000    0.000    0.000 __init__.py:1544(isEnabledFor)
        2    0.000    0.000    0.000    0.000 _weakrefset.py:70(__contains__)
        1    0.000    0.000    0.000    0.000 abc.py:180(__instancecheck__)
        1    0.000    0.000    0.000    0.000 common.py:42(_new_smart_decode)
        2    0.000    0.000    0.000    0.000 compat.py:113(hasattr2)
        3    0.000    0.000    0.000    0.000 compat.py:92(ispython3bytestr)
        3    0.000    0.000    0.000    0.000 compat.py:95(is

Py4JJavaError: An error occurred while calling o55.load.
: com.mongodb.MongoTimeoutException: Timed out after 30000 ms while waiting to connect. Client view of cluster state is {type=UNKNOWN, servers=[{address=192.168.0.20:27017, type=UNKNOWN, state=CONNECTING, exception={com.mongodb.MongoSocketOpenException: Exception opening socket}, caused by {java.net.SocketTimeoutException: connect timed out}}]
	at com.mongodb.internal.connection.BaseCluster.getDescription(BaseCluster.java:177)
	at com.mongodb.internal.connection.SingleServerCluster.getDescription(SingleServerCluster.java:41)
	at com.mongodb.client.internal.MongoClientDelegate.getConnectedClusterDescription(MongoClientDelegate.java:147)
	at com.mongodb.client.internal.MongoClientDelegate.createClientSession(MongoClientDelegate.java:98)
	at com.mongodb.client.internal.MongoClientDelegate$DelegateOperationExecutor.getClientSession(MongoClientDelegate.java:278)
	at com.mongodb.client.internal.MongoClientDelegate$DelegateOperationExec

### Spark withColumn() is a DataFrame function that is used to add a new column to DataFrame, change the value of an existing column, convert the datatype of a column, derive a new column from an existing column.

In [ ]:
FEATURES_COL = ['review_count', 'useful', 'funny', 'cool', 'fans', 'average_stars']
for col in df.columns:
    if col in FEATURES_COL:
        df = df.withColumn(col, df[col].cast('float'))
    else:
        df = df.drop(col)
df = df.na.drop()
df.columns
df.head()

### Spark's implementation of KMeans is a bit different from for example scikit-learn's version. We need to store all features as an array of floats, and store this array as a column called "features". Since we do no longer need the original columns we filter them out with a select statement.

In [ ]:
vecAssembler = VectorAssembler(inputCols=FEATURES_COL, outputCol="features")
df_kmeans = vecAssembler.transform(df)
df_kmeans.show()

In [ ]:
k = 10
kmeans = KMeans().setK(k).setSeed(1).setFeaturesCol("features")
model = kmeans.fit(df_kmeans)
centers = model.clusterCenters()

In [ ]:
transformed = model.transform(df_kmeans)
rows = transformed.collect()
print(rows[:3])

In [ ]:
df_pred = sqlContext.createDataFrame(rows)
df_pred.show()

In [ ]:
pddf_pred = df_pred.toPandas()
pddf_pred.head()

In [ ]:
threedee = plt.figure(figsize=(12,10)).gca(projection='3d')
threedee.scatter(pddf_pred.average_stars, pddf_pred.cool, pddf_pred.fans, c=pddf_pred.prediction)
threedee.set_xlabel('x')
threedee.set_ylabel('y')
threedee.set_zlabel('z')
plt.show()